---

# **Importing of Libraries**

- In this section, we have imported all the libraries that we will used to scrap ali express products.

---

In [4]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

---

# **Ali Express Scraper**

- In this section, we will scrap the items as highlighted in the red box in the following image.

![img](./images/ali-webpage.PNG?raw=true)

- We will use the following user defined function to extract products from ali express website.

---

In [7]:
def main(search, num_of_items):
    """Executes the main program."""

    # Initialize the Chrome driver
    driver = webdriver.Chrome("chromedriver")
    driver.get('https://www.aliexpress.com/')

    # Search the products
    driver.find_element_by_xpath('//*[@id="search-key"]').send_keys(Keys.CONTROL + "a")
    driver.find_element_by_xpath('//*[@id="search-key"]').send_keys(search)
    driver.find_element_by_xpath('//*[@id="search-key"]').send_keys(Keys.RETURN)

    time.sleep(5)

    current_scroll_position = 0
    new_height = 0.8
    sum = 0
    speed = 8
    data_items = num_of_items
    records = list()

    while sum <= data_items:

        # Screen scroller to get similar items
        while current_scroll_position <= new_height:

            current_scroll_position += speed
            driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(0.015)

        # Parse items from start to the scrolled location of the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.findAll('div', attrs={'class': 'JIIxO'})
        for result in results:

            items = result.findAll('div', {'class': '_1OUGS'})
            for item in items:
                try:
                    URL = "https://www.aliexpress.com" + item.find('a', {'class': 'awV9E'})['href']
                except AttributeError:
                    URL = ''

                try:
                    title = item.find('a', {'class': 'awV9E'}).find('span').text
                except AttributeError:
                    title = ''

                try:
                    price = item.find('div', {'class': '_12A8D'}).text
                except AttributeError:
                    price = ''

                try:
                    shipping = item.find('div', {'class': '_2mXVg shIx4'}).find('span').text.strip()
                except AttributeError:
                    shipping = ''

                try:
                    product_location = item.find('a', {'class': '_2lsU7'}).text
                except AttributeError:
                    product_location = ''

                tuple_record = (URL, title, price, shipping, product_location)
                records.append(tuple_record)

            sum = sum + len(items)

        time.sleep(2)
        
        if sum <= data_items:
            button = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[2]/div[2]/div/div[3]/div/div[1]/div/button[2]')
            driver.execute_script("arguments[0].click();", button)

    driver.close()

    # # Saving file to a CSV
    data = pd.DataFrame(data=records, columns=['URL', 'Title', 'Price', 'Shipping', 'Product Location'])
    data.to_csv('results.csv', index=False)
    print('Records saved to the file!, Displaying top 5 records...')
    return data.head()

In [8]:
# Search and get the data associated with the product
search = input("Enter the Product:")
main(search=search, num_of_items=100)

Records saved to the file!, Displaying top 5 records...


,URL,Title,Price,Shipping,Product Location
0,https://www.aliexpress.com/item/10050025937879...,Refurbished Motorola/Motorola V3i Mobile Phone...,US $35.00,Free Shipping,AliExpress Mobile Phone Store
1,https://www.aliexpress.com/item/10050025041985...,Refurbished original 1:1 VERTU mobile phone hi...,US $150.00,+ Shipping: US $2.11,AliExpress Mobile Phone Store
2,https://www.aliexpress.com/item/10050034019812...,"realme C11 2021 RMX3231 6.5"" HD+ Diaplay NFC G...",US $105.00,+ Shipping: US $10.53,iLovemi Store
3,https://www.aliexpress.com/item/10050026871425...,ZANCO Smart Pen World Thinnest Mobile Phone ea...,US $19.99,Free Shipping,Buy Factory Direct Store
4,https://www.aliexpress.com/item/32807294866.ht...,Apple iPhone 5C Used (95%)-Unlocked Original 5...,US $48.87,+ Shipping: US $18.95,Hsenivo World mobile phone Store
